# clio

In [1]:
from bs4 import BeautifulSoup as bs              # 데이터파싱 라이브러리
from selenium import webdriver
import json
import platform
import re
import time
import copy

In [2]:
def getNumber(string):
    numExtracter = re.compile('[0-9]+')
    return int(''.join(numExtracter.findall(string)))
    

In [3]:
def readNextPage():
    html = driver.page_source
    soup = bs(html, 'html.parser')
    pages = soup.find('div',{'class':'paginate'}).find_all('a')
    nextPage= False
    for page in pages:
        if nextPage:
            driver.get(url_products + page['href'])
            nextPage= False
        if page.find('strong'):
            nextPage= True
        
    

In [4]:
def getItemList():
    html = driver.page_source
    soup = bs(html, 'html.parser')
    items = soup.find('div',{'id':'pro_list'}).ul.find_all('li')
    itemList=[]
    for item in items:
        itemList.append('http://www.cliocosmetic.com/ko/product/' + item.a['href'])

    return itemList


In [5]:
def seeDetailInfo():
    html = driver.page_source
    soup = bs(html, 'html.parser')
    seeDetail = soup.find('button',text='구매하러가기')['onclick']
    driver.get(seeDetail[seeDetail.find('http://'):-1])
    

In [13]:
def getItem():
    html = driver.page_source
    soup = bs(html, 'html.parser')

    info = soup.find('div',{'class':'info_area'})

    brandName = '클리오'

    name = info.find('div',{'class':'tit'}).get_text().strip()
    if brandName in name: name = name[len(brandName)+2:].strip()

    image = soup.find('img',{'id':'overimage'})['src']

    category = ''

    volume = soup.find('th',text='용량 또는 중량').parent.td.get_text().strip()

    salePrice = getNumber(info.find('em',{'class':'point'}).get_text())

    originalPrice = info.find('del')
    originalPrice = getNumber(originalPrice.get_text()) if originalPrice else salePrice


    item={}
    item['name'] = name
    item['image']= image
    item['category']=category
    item['volume']=volume
    item['salePrice']=salePrice
    item['originalPrice']=originalPrice
    item['brand']=brandName
    item['url']=driver.current_url

    items=[]
    colorList = soup.find('select',{'id':'O_1'})
    if colorList:
        colors = colorList.find_all('option')
        for color in colors:
            color = color.get_text().strip()
            if '옵션' in color:
                continue
            if '색상' in color:
                continue
            if '선택' in color:
                continue

            if 'NEW' in color:
                color = color[4:].strip()
                
            item_copy = copy.deepcopy(item)
            item_copy['color'] = color
            items.append(item_copy)
    else:
        item['color']=''
        items.append(item)
    display(items)
    return items

In [7]:
def writeJSON(jsonString, output_name='data.json'):
    with open(output_name,'w',encoding='UTF-8') as file:
        file.write(jsonString)


In [8]:
path = 'chromedriver.exe' if (platform.system() == 'Windows') else '/Users/jg/Desktop/develop/DataTeam/DataProcessing/product/crawling/chromedriver';
driver = webdriver.Chrome(path)

url_home = 'http://www.cliocosmetic.com/ko/product/list.asp'
url_products = 'http://www.cliocosmetic.com/ko/product/list.asp'

In [9]:
driver.get(url_products)

itemList = []
while True:
    try:
        itemList += getItemList()
        readNextPage()
    except:
        break
        

In [14]:
result=[]
for item in itemList:
    driver.get(item)
    seeDetailInfo()
    result += getItem()

[{'name': '[어디서든 빛나는 오로라 광채피부]빅 오로라 광채쿠션',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/WM1370QQB9ZDJFEFNREX.jpg',
  'category': '',
  'volume': '25g',
  'salePrice': 25600,
  'originalPrice': 32000,
  'brand': '클리오',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201812267841',
  'color': '002 란제리'},
 {'name': '[어디서든 빛나는 오로라 광채피부]빅 오로라 광채쿠션',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/WM1370QQB9ZDJFEFNREX.jpg',
  'category': '',
  'volume': '25g',
  'salePrice': 25600,
  'originalPrice': 32000,
  'brand': '클리오',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201812267841',
  'color': '003 리넨'},
 {'name': '[어디서든 빛나는 오로라 광채피부]빅 오로라 광채쿠션',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/WM1370QQB9ZDJFEFNREX.jpg',
  'category': '',
  'volume': '25g',
  'salePrice': 25600,
  'originalPrice': 32000,
  'brand': '클리오',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?p

[{'name': '[두드릴수록 살아나는 수분광채]하이드로 메이크업 스폰지 오리지널 마스터 블랜더',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/U1NX9KX0GI4AGKJQ7OB8.jpg',
  'category': '',
  'volume': '1pcs',
  'salePrice': 9000,
  'originalPrice': 9000,
  'brand': '클리오',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201812113376',
  'color': ''}]

[{'name': '★품절대란템★프리즘 에어 섀도우 스파클링',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/L6GJJ2X1KX4ZCC5OEJ1Y.jpg',
  'category': '',
  'volume': '2.3g',
  'salePrice': 9800,
  'originalPrice': 14000,
  'brand': '클리오',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201811132695',
  'color': '016 코랄'},
 {'name': '★품절대란템★프리즘 에어 섀도우 스파클링',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/L6GJJ2X1KX4ZCC5OEJ1Y.jpg',
  'category': '',
  'volume': '2.3g',
  'salePrice': 9800,
  'originalPrice': 14000,
  'brand': '클리오',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201811132695',
  'color': '017 쿠퍼'},
 {'name': '★품절대란템★프리즘 에어 섀도우 스파클링',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/L6GJJ2X1KX4ZCC5OEJ1Y.jpg',
  'category': '',
  'volume': '2.3g',
  'salePrice': 9800,
  'originalPrice': 14000,
  'brand': '클리오',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P20181113269

[{'name': '★18FW 홀리데이★[어드벤처 컬렉션] 프로 플레이 미니브러쉬 세트 [파우치+브러쉬5개구성]',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/G4977I985MTJEI39CAPG.jpg',
  'category': '',
  'volume': '5EA',
  'salePrice': 21000,
  'originalPrice': 30000,
  'brand': '클리오',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201811076799',
  'color': ''}]

[{'name': '★18FW 홀리데이★[어드벤처 컬렉션] 하이드로 메이크업 스폰지(하우스)',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/D1USCKLNAF4G5OP8H78L.jpg',
  'category': '',
  'volume': '4pc',
  'salePrice': 4800,
  'originalPrice': 6000,
  'brand': '클리오',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201811078840',
  'color': ''}]

...

[{'name': '컬링 업 뷰러',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/2(2).jpg',
  'category': '',
  'volume': '1ea',
  'salePrice': 3000,
  'originalPrice': 3000,
  'brand': '클리오',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201304114697',
  'color': ''}]

[{'name': '고급코튼면봉(200개)',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/4.jpg',
  'category': '',
  'volume': '',
  'salePrice': 3000,
  'originalPrice': 3000,
  'brand': '클리오',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201304119960',
  'color': ''}]

[{'name': '네일스타일러1+1',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/B3478CUB4W25UI2H06DH.jpg',
  'category': '',
  'volume': '13ml',
  'salePrice': 5000,
  'originalPrice': 5000,
  'brand': '클리오',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201208245624',
  'color': ''}]

[{'name': '네일 스타일러 리무버',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/NailSTremover.jpg',
  'category': '',
  'volume': '100ml',
  'salePrice': 2000,
  'originalPrice': 2000,
  'brand': '클리오',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201208249573',
  'color': ''}]

[{'name': '쿠션 퍼프(2매)',
  'image': 'http://img.clubclio.co.kr/uploadFolder/WT_PRODUCT/500/DCPVTFKUF0W0AKBHV2PY.jpg',
  'category': '',
  'volume': '2P',
  'salePrice': 5000,
  'originalPrice': 5000,
  'brand': '클리오',
  'url': 'http://www.clubclio.co.kr/product/product_view.asp?product_cd=P201208232134',
  'color': ''}]

In [15]:
output = json.dumps(result,ensure_ascii=False, indent='\t')

writeJSON(output, output_name = 'clio.json')